## Import

In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [3]:
train = pd.read_csv('./train.csv')

## Model Define, Train and Inference

In [ ]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    
    # 학습 데이터 생성
    train_close = train[train['종목코드'] == code][['일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)
    tc = train_close['종가']
    
    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측
    
    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    print(code)
    print(final_return)
    # 결과 저장
    new_df = pd.DataFrame([code, final_return])
    print(new_df)
    results_df = pd.concat([results_df, new_df], ignore_index=True)

In [16]:
new_df = pd.DataFrame(['종목코드':code, "final_return":final_return])
new_df

,0
0,A238490
1,0.010894


In [14]:
#results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df

,종목코드,final_return,0
0,NaN,NaN,A060310
1,NaN,NaN,-0.056272
2,NaN,NaN,A095570
3,NaN,NaN,-0.002972
4,NaN,NaN,A006840
...,...,...,...
3995,NaN,NaN,0.00143
3996,NaN,NaN,A037440
3997,NaN,NaN,0.000169
3998,NaN,NaN,A238490


## Submit

In [6]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [8]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,497
1,A000040,1652
2,A000050,869
3,A000070,1080
4,A000080,1204
...,...,...
1995,A375500,753
1996,A378850,558
1997,A383220,616
1998,A383310,242


In [9]:
baseline_submission.to_csv('baseline_submission.csv', index=False)